In [19]:
import pandas as pd
import numpy as np

In [20]:

#from .autonotebook import tqdm as notebook_tqdm


In [21]:
# load dataset
df = pd.read_csv(r"dataset/synthetic_logs.csv")
df.head()

,timestamp,source,log_message,target_label,complexity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert


In [22]:
df.source.unique()

array(['ModernCRM', 'AnalyticsEngine', 'ModernHR', 'BillingSystem',
       'ThirdPartyAPI', 'LegacyCRM'], dtype=object)

In [23]:
df.target_label.unique()

array(['HTTP Status', 'Critical Error', 'Security Alert', 'Error',
       'System Notification', 'Resource Usage', 'User Action',
       'Workflow Error', 'Deprecation Warning'], dtype=object)

In [24]:
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_distances
from sentence_transformers import SentenceTransformer

# Load the pre-trained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the log messages into embeddings
log_messages = df['log_message'].tolist()
embeddings = model.encode(log_messages)

In [25]:
# Perform clustering using DBSCAN
dbscan = DBSCAN(metric='cosine', eps=0.2, min_samples=1)  # Adjust eps and min_samples as needed
df['cluster'] = dbscan.fit_predict(embeddings)

# Display the cluster assignments
df[['log_message', 'cluster']].head()

,log_message,cluster
0,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,0
1,Email service experiencing issues with sending,1
2,Unauthorized access to data was attempted,2
3,nova.osapi_compute.wsgi.server [req-4895c258-b...,0
4,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,0


In [26]:
df[df['cluster'] == 1]

,timestamp,source,log_message,target_label,complexity,cluster
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,bert,1
217,1/22/2025 5:45,BillingSystem,Mail service encountered a delivery glitch,Error,bert,1
248,5/2/2025 23:04,ModernHR,Service disruption caused by email sending error,Critical Error,bert,1
265,3/30/2025 23:53,ModernCRM,Email system had a problem sending emails,Error,bert,1
361,11/19/2025 23:06,BillingSystem,Email service experienced a sending issue,Error,bert,1
450,10/27/2025 5:59,ThirdPartyAPI,Email delivery system encountered an error,Error,bert,1
477,12/2/2025 10:30,AnalyticsEngine,Email transmission error caused service impact,Critical Error,bert,1
570,11/7/2025 18:08,ThirdPartyAPI,Email service impacted by sending failure,Critical Error,bert,1
678,4/28/2025 15:13,AnalyticsEngine,Email delivery problem affected system,Critical Error,bert,1


In [ ]:
# Group by cluster and sort clusters by the number of records
cluster_counts = df['cluster'].value_counts().sort_values(ascending=False)

# Iterate through the top clusters and print the top 5 records from each
for cluster_id in cluster_counts.index:
    print(f"Cluster {cluster_id} (Count: {cluster_counts[cluster_id]}):")
    print(df[df['cluster'] == cluster_id].head(5))
    print("\n")

Cluster 0 (Count: 1017):
             timestamp         source  \
0  2025-06-27 07:20:25      ModernCRM   
3  2025-07-12 00:24:16       ModernHR   
4  2025-06-02 18:25:23  BillingSystem   
5  2025-10-09 10:30:31       ModernHR   
9  2025-03-30 04:01:45       ModernHR   

                                         log_message target_label complexity  \
0  nova.osapi_compute.wsgi.server [req-b9718cd8-f...  HTTP Status       bert   
3  nova.osapi_compute.wsgi.server [req-4895c258-b...  HTTP Status       bert   
4  nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...  HTTP Status       bert   
5  nova.osapi_compute.wsgi.server [req-f0bffbc3-5...  HTTP Status       bert   
9  nova.osapi_compute.wsgi.server [req-2bf7cfee-a...  HTTP Status       bert   

   cluster  
0        0  
3        0  
4        0  
5        0  
9        0  


Cluster 5 (Count: 147):
              timestamp           source  \
8   2025-02-12 10:42:29    ThirdPartyAPI   
26  2025-03-03 17:11:11        ModernCRM   
40  2025-06